In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy import stats
from src.helpers import *
from src.plot import *
pd.options.display.max_rows = 4000

%matplotlib inline
plt.style.use('ggplot')

## Reading in Dam and 303d data

In [2]:
waters = pd.read_csv('303d_impaired_waters_csv.csv')
dams = pd.read_csv('ca_dams.csv')

FileNotFoundError: [Errno 2] No such file or directory: '303d_impaired_waters_csv.csv'

## Exploratory data: Bar graph of all waterbodies and counts

In [ ]:
plot_water_bodies(body_list, amount_list, 'Number of Water Body Types', (14, 5))

In [ ]:
# a = fill_nans(waters, 'WATER BODY TYPE', 'Null')
# a
body_dict = {}
waters['WATER BODY TYPE'] = waters['WATER BODY TYPE'].fillna('Null')

for body in waters['WATER BODY TYPE']:
    if body in body_dict:
        body_dict[body] += 1
    else:
        body_dict[body] = 1

body_list = []
amount_list = []
for k, v in body_dict.items():
    body_list.append(k)
    amount_list.append(int(v))
    
# print(body_list, amount_list)



## Clean up dfs a little

Cleaning up 303d data

In [ ]:
##csv was from pdf and very messy - drop all rows with NA under Region column
#waters=waters.dropna(axis=0, how='all')


drop_nans(waters, 0, 'all')

##I'm only interested in analyzing river and stream impact - let's filter out all waterbodies that aren't rivers
##or streams
rivers = waters.loc[waters['WATER BODY TYPE'] == 'River & Stream']
#impaired_rivers.count()

In [ ]:
rivers = rivers[['Region', 'Water Body Name', 'POLLUTANT']]
rivers.head()

Cleaning up dam data - dropping unnecessary columns


In [ ]:
## reduce dataframe

dams = dams[['RIVER', 'PRIVATE_DAM']]


## Dealing with NANs in crucial places
There are 90 dams that don't have data for River. We will drop these and maybe manually input data later

In [ ]:
#df = df[df['EPS'].notna()]
dams[dams['RIVER'].isna()].count()

In [ ]:
dams['RIVER'] = dams['RIVER'].str.lower()
rivers['Water Body Name'] = rivers['Water Body Name'].str.lower()

dams = dams[dams['RIVER'].notna()]
dams = dams.loc[dams['RIVER'] != 'offstream']
dams = dams.loc[dams['RIVER'] != 'y']
dams = dams.loc[dams['RIVER'] != '.']
dams = dams.loc[dams['RIVER'] != 3]


In [ ]:
def to_numeric(df, cols):
    for col in cols:
        df[col] = df[col].apply(lambda x: pd.to_numeric(x, errors='coerce'))
    return df

In [ ]:
to_numeric(rivers, ['Region'])
rivers.head()

In [ ]:
rivers.head()

------------------------

## Now we'll group the like-rivers together and append all their polutant and possible sources together. 

In [ ]:
group_by_and_condense(rivers, 'Water Body Name', 
                      'POLLUTANT', 'Water Body Name', 'POLLUTANT')

rivers['All Pollutants'] = rivers[[
    'Water Body Name',
    'Pollutant']].groupby('Water Body Name')['Pollutant'].transform(lambda x: ', '.join(x))

#all individual rivers
rivers = rivers[['Water Body Name', 
                 'All Pollutants', 
                 'Region',]].drop_duplicates()

df['Num Pollutants'] = 0
for i in range (df['All Pollutants'].count()):
    pollutant_list = df['All Pollutants'][i].split(', ')
    pollutant_list = list(dict.fromkeys(pollutant_list))
    df['All Pollutants'][i] = pollutant_list

    df['Num Pollutants'][i] = len(pollutant_list)
    


## Now to match up dammed rivers with impaired rivers

Our goal is to compile a dataframe with columns:

'River', 'Private Dam', 'All Pollutants'


In [ ]:
# impaired = impaired.set_index('Water Body Name', drop=False)
# impaired = impaired.rename(columns={'Water Body Name': 'Merger'})

# cal_dams = cal_dams.set_index('RIVER', drop=False)
# cal_dams = cal_dams.rename(columns={'RIVER': 'Merger'})






In [ ]:
# for impaired_river in impaired['Merger']:
    
#     for river in cal_dams['Merger']:
#         if river in impaired_river:
#             impaired['Merger'].loc[impaired['Merger'] == impaired_river] = river
#             impaired_river = river
#             break

            

In [ ]:
## Condense cal_dams to only one entry per river

# s_cal_dams = cal_dams.groupby(['Merger'], sort=False)['PRIVATE_DAM'].apply(lambda x: ','.join(x.astype(str)))


# Merging impaired and cal_dams to one df
df = all impaired rivers with associated dam values

In [ ]:
#df = impaired.merge(s_cal_dams, how='left', left_on='Merger', right_on='Merger')
df = merge_dfs(rivers, dams, 'River', 'River')

# PLOTTING TIME


## Bar Chart of #dammed vs. #undammed

In [ ]:
df_dammed = df[df['PRIVATE_DAM'].notnull()]
natural = df.loc[df['PRIVATE_DAM'].isnull()]


In [ ]:
num = [len(df_dammed), len(df_not_dammed)]
labels = ['Undammed', 'Dammed']
explode = (0, 0.08)

fig1, ax1 = plt.subplots(figsize=(5.5,5.5))
ax1.pie(num, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.set_title('Pergentage of Dammed and Undammed Impaired Rivers')

Let's make a histogram for # of pollutants in dammed and undammed rivers:

In [ ]:
df_dammed.index = np.arange(0, len(df_dammed))
df_not_dammed.index = np.arange(0, len(df_not_dammed))

fig, ax = plt.subplots()
#df_dammed.hist('Num Pollutants', ax=ax, bins=20, bottom=0.1, alpha=.5, color='black')
df_not_dammed.hist('Number of Pollutants', ax=ax, bins=20, bottom=0.1, alpha=.5)

In [ ]:
fig, ax = plt.subplots()
df_dammed.hist('Num Pollutants',ax=ax, bins=20, bottom=0.1, alpha=.5, color='black')

Now we should plot individual pollutants on the x-axis, and a stacked bar chart on top!

1: make a list(maybe dictionary?) of all pollutants

2: 

A/B Test: Dammed Rivers have more pollutants than non-dammed rivers:
ho: Dammed rivers and un_dammed rivers have the same average number of pollutants
ha: dammed rivers have more pollutants than undammed rivers



In [ ]:
A/B Test:
    ho: dammed rivers and undammed rivers have the same distribution of impairments
    ha: they have a different distribution of impairments

In [ ]:
df_dammed.head()

In [ ]:
#df_dammed:
n = df_dammed['Num Pollutants'].count()   
p = sum([x for x in df_dammed['Num Pollutants']])/n
p
#binomial = stats.binom(n=3100, p=0.80)

In [ ]:
def bootstrap_sample_means(data, n_bootstrap_samples=10000):
    bootstrap_sample_means = []
    for i in range(n_bootstrap_samples):
        bootstrap_sample = np.random.choice(data, size=300, replace=True)  ##changed from size=len(data)
        bootstrap_sample_means.append(np.mean(bootstrap_sample))
    return bootstrap_sample_means

In [ ]:
dammed_means = bootstrap_sample_means(df_dammed['Num Pollutants'])


fig, ax = plt.subplots(1, figsize=(10, 4))
_ = ax.hist(df_dammed['Num Pollutants'], bins=25, density=True, color="black", alpha=0.4,
            label="Sample Data")
_ = ax.hist(dammed_means, bins=25, density=True, color="red", alpha=0.75,
            label="Bootstrap Sample Means")
ax.legend()
_ = ax.set_title("Bootstrap Sample Means Dammed Rivers (10000 samples)")

# Need to normalize based on sample size!

In [ ]:
## dammed means vs undammed means

dammed_means = bootstrap_sample_means(df_dammed['Num Pollutants'])
undammed_means = bootstrap_sample_means(df_not_dammed['Num Pollutants'])


fig, ax = plt.subplots(1, figsize=(10, 4))
_ = ax.hist(dammed_means, bins=20, density=True, color="blue", alpha=0.4,
            label="Dammed Rivers")
_ = ax.hist(undammed_means, bins=20, density=True, color="red", alpha=0.75,
            label="Undammed Rivers")
ax.legend()
_ = ax.set_title("Mean number of Pollutants in Dammed and Undammed Rivers")

# Plot stacked bar graph

In [ ]:
n = 19
# 




In [ ]:
keys = dammed_most_potent.keys()
values = dammed_most_potent.values()

fig, ax = plt.subplots(figsize=(16,5))
plt.bar(keys, values)



N = 5
menMeans = (20, 35, 30, 35, 27)
womenMeans = (25, 32, 34, 20, 25)
menStd = (2, 3, 4, 1, 2)
womenStd = (3, 5, 2, 3, 3)
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, menMeans, width, yerr=menStd)
p2 = plt.bar(ind, womenMeans, width,
             bottom=menMeans, yerr=womenStd)

plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(ind, ('G1', 'G2', 'G3', 'G4', 'G5'))
plt.yticks(np.arange(0, 81, 10))
plt.legend((p1[0], p2[0]), ('Men', 'Women'))

plt.show()

In [ ]:
df_dammed.head()

In [ ]:
dammed_pollutant_dict = {}
for pol in df_dammed['All Pollutants']:
    for pollutant in pol:
        if pollutant in dammed_pollutant_dict:
            dammed_pollutant_dict[pollutant] += 1
        else:
            dammed_pollutant_dict[pollutant] = 1
#for k, v in dammed_pollutant_dict.items():
#    print(k, v)
dammed_most_potent = {k:v for (k,v) in dammed_pollutant_dict.items() if v > 20}
len(dammed_most_potent)

In [ ]:
df_dammed['All Pollutants'][3]
len(df_dammed)

In [ ]:
undammed_pollutant_dict = {}
for pol in df_not_dammed['All Pollutants']:
    for pollutant in pol:
        if pollutant in undammed_pollutant_dict:
            undammed_pollutant_dict[pollutant] += 1
        else:
            undammed_pollutant_dict[pollutant] = 1

        
            

#twentyfive most potent
undammed_most_potent = {k:v for (k,v) in undammed_pollutant_dict.items() if v > 28}

len(undammed_most_potent)

# Agricultural = ['Diazinon', 'Pyrethroids', 'Nitrate', 
#                 'Chlorpyrifos', 'DDT (Dichlorodiphenyltrichloro ethane)',
#                'Dieldrin', 'Nitrogen', 'Total Nitrogen as N']
# Metals = ['Copper', 'Zinc', 'Lead', 'Iron']

In [ ]:
print(undammed_pollutant_dict['Mercury'])
print(dammed_pollutant_dict['Mercury'], dammed_pollutant_dict['Unknown Toxicity'])

sum = 0
for k, v in undammed_pollutant_dict.items():
    sum += v
16/sum

# Plot bars side by side

In [ ]:
N = 5
men_means = (20, 35, 30, 35, 27)
women_means = (25, 32, 34, 20, 25)

ind = np.arange(N) 
width = 0.35       
plt.bar(ind, men_means, width, label='Men')
plt.bar(ind + width, women_means, width,
    label='Women')

plt.ylabel('Scores')
plt.title('Scores by group and gender')

plt.xticks(ind + width / 2, ('G1', 'G2', 'G3', 'G4', 'G5'))
plt.legend(loc='best')
plt.show()

In [ ]:
x_list = []
for k, v in dammed_most_potent.items():
    if k not in x_list:
        x_list.append(k)
    
for k, v in undammed_most_potent.items():
    if k not in x_list:
        x_list.append(k)
x_list = sorted(x_list)

dammed = []
for item in x_list:
    if item in dammed_most_potent:
        dammed.append(dammed_most_potent[item])
    else:
        dammed.append(0.5)
undammed = []
for item in x_list:
    if item in undammed_most_potent:
        undammed.append(undammed_most_potent[item])
    else:
        undammed.append(0.5)
        
fig, ax = plt.subplots(figsize=(20, 5))
        
N = len(x_list)
ind = np.arange(N)
width = 0.4
      
ax.bar(ind, dammed, width, label='Dammed')
ax.bar(ind + width , undammed, width,
      label='Undammed')

plt.ylabel('# of Occurances of Impairment')
plt.title('Top 15 Impairments in Dammed and Undammed Rivers')

plt.xticks(ind + width / 2, x_list)
plt.setp(plt.gca().get_xticklabels(), rotation=30, horizontalalignment='right')
plt.legend(loc='best')
plt.show()

In [3]:
d_keys = dammed_most_potent.keys()
d_values = dammed_most_potent.values()

fig, ax = plt.subplots(figsize=(25,5))
plt.bar(d_keys, d_values)

NameError: name 'dammed_most_potent' is not defined

In [ ]:
u_keys = undammed_most_potent.keys()
u_values = undammed_most_potent.values()

fig, ax = plt.subplots(figsize=(25,5))
plt.bar(u_keys, u_values, color='red')

## let's go through pollutants and classify them as 'direct human addition' or 'change of environmental condition'

In [ ]:
sum=0
for k, v in undammed_most_potent.items():
    sum += v
    print(k, v)
print(sum)

## Classifying top 15 pollutants as either 'Yes' or 'No' if they are TMDL solvable

In [ ]:
treatable = {}
treatable['Diazinon'] = 'Yes'
treatable['Sedimentation/Siltation'] = 'No'
treatable['Pathogens'] = 'Yes'
treatable['Low Dissolved Oxygen'] = 'No'
treatable['Fecal Coliform'] = 'Yes'
treatable['Nitrate'] = 'Yes'
treatable['Sediment Toxicity'] = 'No'
treatable['Unknown Toxicity'] = 'No'
treatable['Chlorpyrifos'] = 'Yes'
treatable['Escherichia coli (E. coli)'] = 'Yes'
treatable['pH'] = 'No'
treatable['Chloride'] = 'Yes'
treatable['Turbidity'] = 'No'
treatable['Pathogens'] = 'No'
treatable['Selenium'] = 'Yes'
treatable['Temperature'] = 'No'
treatable['Mercury'] = 'No'
treatable['Copper'] = 'Yes'
treatable['Total Dissolved Solids'] = 'No'
treatable['Trash'] = 'Yes'
treatable['Toxicity'] = 'No'
treatable['water'] = 'No'


### Undammed %solvable

In [ ]:
undammed_solvable = 0
undammed_unsolvable = 0
for k, v in undammed_most_potent.items():
    if he[k] == 'Yes':
        undammed_solvable += v
    else:
        undammed_unsolvable += v
print(undammed_solvable, undammed_unsolvable)

### Dammed %unsolvable

In [ ]:
def solvability(most_potent_dict, he):
    sum=0
    solvable = 0
    unsolvable = 0
    for k, v in most_potent_dict.items():
        if he[k] == 'Yes':
            solvable += v
        else:
            unsolvable += v
        sum += v
    return [solvable, unsolvable]


### solvable vs. unsolvable pie chart

In [ ]:

dammed_vs_natural = [5, 7]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
for s, ax in zip(dammed_vs_natural, axs.flatten()):
    ax.axis('equal')
    ax.pie(solvability(s, treatable), explode=explode, labels=labels, colors=['steelblue', 'goldenrod'], autopct='%1.1f%%',
        shadow=True, startangle=90)

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = ['Solvable with TMDLs', 'Not Feasibly Solved with TMDLs']
solvable = solvability(undammed_most_potent, he)
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots(figsize=(4.5, 4.5))
ax1.pie(solvable, explode=explode, labels=labels, colors=['steelblue', 'goldenrod'], autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.set_title('TMDL Feasibility for Undammed Rivers')
plt.show()

In [ ]:
labels = ['Solvable with TMDLs', 'Not Feasibly Solved with TMDLs']
solvable = solvability(dammed_most_potent, he)
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots(figsize=(4.5,4.5))
ax1.pie(solvable, explode=explode, labels=labels, colors=['steelblue', 'goldenrod'], autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.set_title('TMDL Feasibility for Dammed Rivers')
plt.show()

In [ ]:
sum_d=0
for k, v in dammed_most_potent.items():
    sum_d += v
    print(k, v)
print(sum_d)

In [ ]:
he

Run an A/B test to see whether there is a difference in the makeup of 'TMDL Treatable' vs. 'Environmental' for dammed and undammed!

Ho: Both undammed and dammed have same proportion of TMDL treatable to environmental impairements
Ha: dammed and undammed have a different proportion of TMDL to Environmental impairments

